In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
from sklearn.metrics import r2_score


#from keras.utils.vis_utils import model_to_dot
#from keras.utils import plot_model
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


from IPython.display import Image
import glob
import cv2
import random


In [5]:
my_Path = 'train_val/'

files = list(glob.glob(my_Path+'*.jpg'))
files
len(files)
files[0:5]

['train_val\\100_1_0_20170112215032192.jpg.chip.jpg',
 'train_val\\100_1_2_20170112213615815.jpg.chip.jpg',
 'train_val\\10_0_0_20161220222308131.jpg.chip.jpg',
 'train_val\\10_1_0_20170109203245653.jpg.chip.jpg',
 'train_val\\116_1_3_20170120134744096.jpg.chip.jpg']

In [15]:
img = cv2.imread(files[1])

In [9]:
img.shape

(128, 128, 3)

In [12]:
df = pd.read_csv('df.csv', index_col= 0)
                 
                 


In [13]:
df.head(2)

,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,...,p_49144,p_49145,p_49146,p_49147,p_49148,p_49149,p_49150,p_49151,Gender,Age
0,14,16,17,34,37,41,85,89,94,117,...,32,30,21,29,28,7,15,14,1,99
1,154,151,173,154,152,174,155,152,178,152,...,236,231,197,191,184,149,142,133,0,87


In [17]:
X   = df.drop(['Age', 'Gender'], axis = 1)
y = df['Gender']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

In [20]:
image_rows = 128
image_cols = 128
batch_size = 4096


image_shape = (image_rows,image_cols,3)
train_data = np.array(X_train, dtype = 'float32')
test_data = np.array(X_test, dtype='float32')


#x_validate = x_validate.reshape(x_validate.shape[0],*image_shape)

In [23]:
len(train_data[0])

49152

In [ ]:



x_train = X_train.reshape(X_train.shape[0],*image_shape)
x_test = X_test.reshape(X_test.shape[0],*image_shape)